In [3]:
from IPython.display import HTML
HTML('''<style>.CodeMirror{min-width:100% !important;}</style>''')

<style>
.container {width: 99% !important; }
.text_cell {width: 100% !important; }
</style>


# This is a test
## Yo, does this work?
Does not look like it :(